In [36]:
# Cubic (10 runs) mean fitness: 252
Cubic_pso_config_mean = [65.07982407, 4.95690714, 0.28949701, 0.54421663, 0.39890445, 1.42315492, 1.18854163]

# Linear (10 runs) mean fintess: 9.17486598344206e+24
Linear_pso_config_mean = [75.13534983, 6.59595661, 0.15486748, 0.63274028, 0.93549642, 1.5667033, 0.48415203]

# Tanh (10 runs) mean fitness: 23659 
Tanh_pso_config_mean = [54.98334367, 6.41212707, 0.59507185, 1.23273439, 1.68258425, 1.01222575, 0.19820747]

In [37]:
# Cubic Most fit model: 6847
Cubic_ann_params = np.array([ 4.06948631e+00, -7.29125756e-01,  5.73721712e-01, -4.97778569e-01, 1.21867021e-01, -4.07830431e-01,  9.23118308e-01,  9.96763804e-01, -3.53176479e-01,  2.82162360e+00, -6.77411971e-01, -6.33183382e-01, 3.17935306e-01, -1.91733120e-01,  2.57710600e-03])

#Linear Most fit model: 9.17486598344206e+25
Linear_ann_params = np.array([0.33653903, 0.62395429, 0.38036724,-0.73197691, 0.27957089, -0.44447831, 0.90125204, -0.72400064, -0.73325263, 0.2053941, -0.37270846, 0.1568939, 0.4157871, -0.37861424, -0.57400409])

#Sine Most fit model: 329
Sine_ann_params = np.array([ 3.50680037,  0.2952162 ,  0.71664711, -0.95665899, -0.9055195, -0.27101866,  0.41973435,  0.36889121, -0.28896977,  3.3595307, 0.42623047,  0.30804742,  0.62065839, -0.73504326,  0.70304275])

#Tanh Most fit model: 20498
Tanh_ann_params = np.array([ 4.30993349,  0.2291253 , -0.99626464, -0.29776704,  0.99032436, 0.54519926, 0.98178641, -0.06046016, 0.9675356 ,  2.17890815, -0.49269196, -0.29330431, -0.94045692, 0.72556016, 0.95398864])



In [38]:
import numpy as np
from Coursework.ANNModel import model as mod
from Coursework.ANNModel import data as d
from Coursework.PSO import pswarm as ps
from Coursework.PSO import psobehaviour as psh
from Coursework.PSO import history as hs
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

data = d.Data("./Data/1in_linear.txt")
decimals = 'linear'
X = data.get_rows()
y = data.get_output()

psoParams = Linear_pso_config_mean
annParams = Linear_ann_params

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=14)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
#train the model
model = mod.ANN()
model.set_training_input(X_train, y_train)
model.add(mod.Layer(4, activation="sigmoid"))
model.add(mod.Layer(1, activation="null"))
model.compile()

model_history = hs.PSOHistory(model)

Model Compiled!


In [40]:
pso = ps.PSO(swarm_size=100, num_informants=6, boundary_policy=psh.BoundaryPolicy.RANDOMREINIT, termination_policy=[psh.TerminationPolicy.ITERATIONS], termination_args={'max_iter': 400}, alpha=0.61, beta=1.3, gamma=1.4, delta=1.3, epsilon=0.6)
pso = pso.decode_vec(psoParams)

print(pso.swarm_size)

75


In [41]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)

train ={
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

test = {
    'scoreA' : [],
    'scoreB' : [],
    'fitnessList': [],
    'lossList': []
}

for i in range(10):
    fitness = pso.run()
    # Evaluating on training set
    model.set_training_input(X_train, y_train)
    train['fitnessList'].append(model.evaluate_fitness(pso.best.location))
    score = data.score(model, decimals, atol=0.07)
    train['scoreA'].append(score[0])
    train['scoreB'].append(score[1])
    train['lossList'].append(model.loss)
    # Evaluating on testing set
    model.set_training_input(X_test, y_test)
    test['fitnessList'].append(model.evaluate_fitness(pso.best.location))
    score = data.score(model, decimals, atol=0.07)
    test['scoreA'].append(score[0])
    test['scoreB'].append(score[1])
    test['lossList'].append(model.loss)

print("Testing on train - A: ", np.mean(train['scoreA']), " B: ", np.mean(train['scoreB']), " Fitness: ", round(np.mean(train['fitnessList']), 3), " Loss: ", round(np.mean(train['lossList']), 3))
print("Testing on test - A: ", np.mean(test['scoreA']), " B: ", np.mean(test['scoreB']), " Fitness: ", round(np.mean(test['fitnessList']), 3), " Loss: ", round(np.mean(test['lossList']), 3))

Testing on train - A:  0.6820895522388061  B:  0.6820895522388061  Fitness:  352.099  Loss:  0.012
Testing on test - A:  0.7181818181818183  B:  0.7181818181818183  Fitness:  360.638  Loss:  0.011


In [42]:
pso.set_search_dimensions(model.dimension_vec())
pso.set_fitness_fn(model_history.evaluate_fitness)
fitness = pso.run()
print(fitness.location)
print(fitness.fitness)

[ 0.31957361  0.71919737 -0.24190541  0.6168814   0.15890638 -0.14500973
 -0.96611625  0.23984413  0.02269652  2.11737217 -0.14210612 -0.27407302
 -0.87264901  0.17514957  0.11068013]
76.2519306336005


In [43]:
print('activation: ', model.layers[-1].activation)
print('bias: ', model.layers[-1].bias)
print('weights: ', model.layers[-1].weights)

activation:  ActivationFunction.HYPERBOLIC_TANGENT
bias:  [-0.14210612]
weights:  [[-0.27407302]
 [-0.87264901]
 [ 0.17514957]
 [ 0.11068013]]


In [44]:
model.decode_vec(annParams)
model.one_pass()

# Evaluating on training set
model.set_training_input(X_train, y_train)
train['fitnessList'].append(model.evaluate_fitness(annParams))
score = data.score(model, decimals, atol=0.07)
train['scoreA'].append(score[0])
train['scoreB'].append(score[1])
train['lossList'].append(model.loss)
# Evaluating on testing set
model.set_training_input(X_test, y_test)
test['fitnessList'].append(model.evaluate_fitness(annParams))
score = data.score(model, decimals, atol=0.07)
test['scoreA'].append(score[0])
test['scoreB'].append(score[1])
test['lossList'].append(model.loss)

print("Testing on train - A: ", np.mean(train['scoreA']), " B: ", np.mean(train['scoreB']), " Fitness: ", round(np.mean(train['fitnessList']), 3), " Loss: ", round(np.mean(train['lossList']), 3))
print("Testing on test - A: ", np.mean(test['scoreA']), " B: ", np.mean(test['scoreB']), " Fitness: ", round(np.mean(test['fitnessList']), 3), " Loss: ", round(np.mean(test['lossList']), 3))


Testing on train - A:  0.7109905020352782  B:  0.7109905020352782  Fitness:  2.7318633230872164e+16  Loss:  0.011
Testing on test - A:  0.7438016528925622  B:  0.7438016528925622  Fitness:  3.0729444178828092e+16  Loss:  0.01


In [45]:
1/model.loss

3.380238859671054e+17

In [46]:
model.evaluate_fitness(Cubic_ann_params)

14.701404307863989